In [1]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Anjali\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anjali\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from nltk.stem import  WordNetLemmatizer
lemmatizer =  WordNetLemmatizer()

In [3]:
from nltk.corpus import wordnet
from nltk import pos_tag
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN      
    

In [4]:
from nltk.corpus import stopwords
import string 
#stops = set(stopwords.words('english'))
stops = list(string.punctuation)
stops

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [6]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
import csv
import string 
from nltk.corpus import stopwords
class TwitterClient(object): 

    def __init__(self): 
        consumer_key = "rXBTXz6yE6VjactwIFfQEHLwX"
        consumer_secret = "3wluIGBoyitUyr8m0HRhiap6UhUFkGskWJ2DA0J7v4es9Dfhwo"
        access_token = "1055057699078107137-ntZ1ReqvsUvpPDJpKjUaV5oc0m8zhb"
        access_token_secret = "39Ew12bFgpyITIjo9GOhvD2R3mlpwvrbvAaCE5XmGQYm4"


        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 

    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())         
        
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 1
        elif analysis.sentiment.polarity == 0: 
            return 0
        else: 
            return -1

    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 

        try: 
            # call twitter api to fetch tweets 
            file_name = open("test.txt","r") 
            train = "train.csv"
            csvf = open("train.csv","w")
            csvwriter = csv.writer(csvf)
            row = ["text","sentiment"]
            csvwriter.writerow(row)
            for line in file_name:
                fetched_tweets = self.api.search(q = line, count = count) 
  
                # parsing tweets one by one 
                for tweet in fetched_tweets: 
                    # empty dictionary to store required params of a tweet 
                    parsed_tweet = {} 
  
                    # saving text of tweet 
                    parsed_tweet['text'] = tweet.text 
                    # saving sentiment of tweet 
                    parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

                    # appending parsed tweet to tweets list 
                    if tweet.retweet_count > 0: 
                        # if tweet has retweets, ensure that it is appended only once 
                        if parsed_tweet not in tweets: 
                            tweets.append(parsed_tweet)
                            
                    else: 
                        tweets.append(parsed_tweet)
                    #tweets is our list 
                    f=0
                    val1 = ""
                    val2 = ""
                    for i in tweets:
                        for key, value in i.items():
                            if(f==0):
                                val1=value
                                f=1
                            else :
                                f=0
                                val2=value
                        val1 = val1.replace(',', '')
                        val3 = ' '.join([word for word in val1.split() if word not in  stopwords.words("english")])
                       # stops = set(stopwords.words('english'))
                        row = [val3.encode("utf8"),val2]
                        csvwriter.writerow(row)  
            #csvf.close()
            return tweets 
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 

def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Donald Trump', count = 200) 

    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 1] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == -1] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    #print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 

    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 

    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 

if __name__ == "__main__": 
    # calling main function 
    main() 


Positive tweets percentage: 29.26829268292683 %
Negative tweets percentage: 22.172949002217294 %


Positive tweets:
@djcarlile @JPFinlayNBCS The defense looks for real though and they've faced some good QBs so far and have a 5-2 re… https://t.co/XR563Rbfz2
RT @5liveSport: On Sports Extra, more #CarabaoCup action tonight

⚽ @NorwichCityFC eliminated @afcbournemouth on the only other occasion th…
On Sports Extra, more #CarabaoCup action tonight

⚽ @NorwichCityFC eliminated @afcbournemouth on the only other occ… https://t.co/XAww6STLtf
@nytimes Anything the POTUS has done has faced legal opposition, even though most have failed.  The left has made t… https://t.co/83HvxNm7R5
@SuzanneWaldman This "conflict &amp; drama" is barely half a decade old, and largely coincides with the Arab Spring and… https://t.co/GrbJVV4xsS
2.25: Due to the long delay of my luggage, I faced much inconveniences. Was really reluctant to spend just cos of i… https://t.co/BDPneuZLAm
The IN Democratic Party is endorsi

In [7]:
import pandas as pd
df=pd.read_csv("train.csv");
df1 = df
df1


,text,sentiment
0,b'@tfwrail 2/2 managerial team ran ariva train...,-1
1,b'@tfwrail 2/2 managerial team ran ariva train...,-1
2,b'RT @aruna_sekhar: @IndEditorsGuild Notes sta...,0
3,b'@tfwrail 2/2 managerial team ran ariva train...,-1
4,b'RT @aruna_sekhar: @IndEditorsGuild Notes sta...,0
5,b'RT @cosmexbox: \xe0\xb8\x9e\xe0\xb8\xa3\xe0\...,0
6,b'@tfwrail 2/2 managerial team ran ariva train...,-1
7,b'RT @aruna_sekhar: @IndEditorsGuild Notes sta...,0
8,b'RT @cosmexbox: \xe0\xb8\x9e\xe0\xb8\xa3\xe0\...,0
9,b'Shrewsbury accident emergency department ful...,-1


In [8]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
y = df['sentiment']
x = df['text']
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['sentiment'] , test_size=0.2)

In [9]:
TRAIN_PROCESSED_FILE = 'train-processed.csv'
TEST_PROCESSED_FILE = 'test-processed.csv'
TEST_PROCESSED_FILE_CHECK = 'test-processed-check.csv'
X_test.to_csv(TEST_PROCESSED_FILE)


In [10]:
final_train = pd.concat([X_train, y_train,], axis=1)

In [11]:
final_train.to_csv(TRAIN_PROCESSED_FILE)

In [ ]:
import utils

# Classifies a tweet based on the number of positive and negative words in it
POSITIVE_WORDS_FILE = 'positive-words.txt'
NEGATIVE_WORDS_FILE = 'negative-words.txt'
TRAIN = True

def funct(l):
    if(l=="negative"):
        return -1
    elif(l=="positive"):
        return 1
    else :
        return 0
def classify(processed_csv, test_file=True, **params):
    positive_words = utils.file_to_wordset(params.pop('positive_words'))
    negative_words = utils.file_to_wordset(params.pop('negative_words'))
    predictions = []
    with open(processed_csv, 'r') as csv:
        for line in csv:
            if test_file:
                tweet_id, tweet = line.strip().split(',')
            else:
                tweet_id,tweet,label= line.strip().split(',')
            pos_count, neg_count = 0, 0
            for word in tweet.split():
                if word in positive_words:
                    pos_count += 1
                elif word in negative_words:
                    neg_count += 1
            # print pos_count, neg_count
            prediction = 1 if pos_count >= neg_count else 0
            if test_file:
                predictions.append((tweet_id, prediction))
            else:
                predictions.append((tweet_id, funct(label), prediction))
    return predictions

if __name__ == '__main__':
    if TRAIN:
        predictions = classify(TRAIN_PROCESSED_FILE, test_file=(not TRAIN), positive_words=POSITIVE_WORDS_FILE, negative_words=NEGATIVE_WORDS_FILE)
        correct = sum([1 for p in predictions if p[1] == p[2]]) * 100.0 / len(predictions)
        print('Correct = %.2f%%' % correct)
    else:
        predictions = classify(TEST_PROCESSED_FILE, test_file=(not TRAIN), positive_words=POSITIVE_WORDS_FILE, negative_words=NEGATIVE_WORDS_FILE)
        utils.save_results_to_csv(predictions, 'baseline.csv')

In [ ]:
FREQ_DIST_FILE = "twitter-sentiment-analysis/code/train-processed-freqdist.pkl"
unigrams = utils.top_n_words(FREQ_DIST_FILE, 100)
FREQ_DIST_FILE_bi = "twitter-sentiment-analysis/code/train-processed-freqdist-bi.pkl"
bigrams = utils.top_n_words(FREQ_DIST_FILE_bi, 100)


In [ ]:
USE_BIGRAMS = True
def get_feature_vector(tweet):
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in range(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


In [ ]:

def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)

    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file

    Returns:
        list: Of tuples
    """
    tweets = []
    print('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id,  tweet, sentiment = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, sentiment, feature_vector))
            utils.write_status(i + 1, total)
    print('\n')
    return tweets


In [ ]:
test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)

In [ ]:
train_tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)

In [ ]:
import ast
for i,tweet in enumerate(train_tweets):
    if i!=0:
        f = 1.0
        f = float(tweet[1])
        print(f)
    #print(ast.literal_eval(tweet[1]))
    

In [ ]:
for i,tweet in enumerate(test_tweets):
    print(tweet[1][1])
    

In [ ]:
with open(TRAIN_PROCESSED_FILE, 'r') as csv:
        lines = csv.readlines()
        #print(lines)
        total = len(lines)
        for i, line in enumerate(lines):
            line = line.strip().split(',')
            print(i)
            print(line[2])